In [0]:
!wget http://perso-etis.ensea.fr/picard/Sarcasm_Headlines_Dataset.json

--2018-11-10 17:01:23--  http://perso-etis.ensea.fr/picard/Sarcasm_Headlines_Dataset.json
Resolving perso-etis.ensea.fr (perso-etis.ensea.fr)... 193.51.45.246
Connecting to perso-etis.ensea.fr (perso-etis.ensea.fr)|193.51.45.246|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://perso-etis.ensea.fr//picard/Sarcasm_Headlines_Dataset.json [following]
--2018-11-10 17:01:24--  https://perso-etis.ensea.fr//picard/Sarcasm_Headlines_Dataset.json
Connecting to perso-etis.ensea.fr (perso-etis.ensea.fr)|193.51.45.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643544 (5.4M) [application/json]
Saving to: ‘Sarcasm_Headlines_Dataset.json.1’

Sarcasm_Headlines_D 100%[===================>]   5.38M  7.19MB/s    in 0.7s    

2018-11-10 17:01:25 (7.19 MB/s) - ‘Sarcasm_Headlines_Dataset.json.1’ saved [5643544/5643544]



In [0]:
import json
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [0]:
j = json.load(open('Sarcasm_Headlines_Dataset.json'))

headlines = [i['headline'] for i in j]

#Suppression des majuscules ainsi que des caractères spéciaux
headlines = [keras.preprocessing.text.text_to_word_sequence(i['headline'],filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\' ', lower=True, split=' ') for i in j]
print(headlines[0])
print(len(headlines))

labels = np.array([i['is_sarcastic'] for i in j])

print(f'{len(headlines)} headlines of which {np.mean(labels)} are sarcastic')

['former', 'versace', 'store', 'clerk', 'sues', 'over', 'secret', 'black', 'code', 'for', 'minority', 'shoppers']
26709
26709 headlines of which 0.43895316185555433 are sarcastic


**Tokenization**

Dans un premier temps, nous allons construire un dictionnaire qui associe un entier à chaque mot (par exemple, les 10k mots les plus fréquents). Pensez à réserver un entier pour les mots non encodés (1 par exemple) et un entier pour l'absence de mot (0 par exemple).

Ensuite, nous allons convertir une phrase en une séquence d'entier en utilisant le dictionnaire. Puis nous allons transformer les phrases en vecteurs de taille fixe par zero padding.


In [0]:
#Création des dictionnaires
occurrence = dict()
dictionary = dict()

N=0 #Nombre des mots

#Comptage du nombre d'occurences de chaque mots dans les headlines
for headline in headlines:
  for words in headline:
    N+=1
    try:
      occurrence[words] += 1
    except KeyError:
      occurrence[words] = 1
     
print(str(len(occurrence)) + ' unique words in dataset, ' + str(N) + ' words analyzed')

#On construit le dictionnaire des 10k mots les plus fréquents
dictionary['']=0 #Absence de mots
dictionary[' ']=1 #Mot non encodé
compt = 2
for compt, words in enumerate(sorted(occurrence, key=occurrence.get, reverse = True)):
  if compt> 1e4:
    break;
  else :
    dictionary[words]=compt
    compt += 1
  
  
print(dictionary)

25392 unique words in dataset, 274609 words analyzed
{'': 0, ' ': 1, 'to': 0, 'of': 1, 'the': 2, 's': 3, 'in': 4, 'for': 5, 'a': 6, 'on': 7, 'and': 8, 'with': 9, 'trump': 10, 'is': 11, 'new': 12, 'man': 13, 'from': 14, 'at': 15, 'you': 16, 't': 17, 'it': 18, 'about': 19, 'this': 20, 'by': 21, 'after': 22, 'up': 23, 'out': 24, 'be': 25, 'that': 26, 'how': 27, 'he': 28, 'as': 29, 'not': 30, 'what': 31, 'can': 32, 'are': 33, 'your': 34, 'his': 35, 'who': 36, 'all': 37, 'just': 38, 'has': 39, 'will': 40, 'year': 41, 'more': 42, 'one': 43, 'into': 44, 'report': 45, 'i': 46, 'why': 47, 'have': 48, 'area': 49, 'over': 50, 'woman': 51, 'donald': 52, 'day': 53, 'u': 54, 'says': 55, 'time': 56, 'first': 57, 'obama': 58, 'no': 59, 'like': 60, 'women': 61, 'people': 62, 'get': 63, 'her': 64, 'old': 65, 'world': 66, 'off': 67, 'we': 68, 'life': 69, 'an': 70, 'nation': 71, 'now': 72, 'house': 73, 'clinton': 74, 'they': 75, 'make': 76, 'still': 77, 'white': 78, 'back': 79, 'was': 80, 'than': 81, 'my'

In [0]:
#Encoding des phrases
length_max = len(max(headlines, key = len)) #Longeur de la headline la plus longue
print(length_max)

def encode(sentence,dictionary):
  try:
    words = sentence.split(' ')
  except AttributeError:
    words = sentence
  encoded = np.zeros((1,length_max))
  for i in range(len(words)):
    try:
        encoded[0,i]=dictionary[words[i]];
    except KeyError:
        encoded[0,i]=1;
  return encoded
      
print(headlines[0])
encode(headlines[0],dictionary)

39
['former', 'versace', 'store', 'clerk', 'sues', 'over', 'secret', 'black', 'code', 'for', 'minority', 'shoppers']


array([[3.140e+02, 1.000e+00, 6.560e+02, 3.164e+03, 2.329e+03, 5.000e+01,
        3.730e+02, 9.500e+01, 2.123e+03, 5.000e+00, 2.627e+03, 8.262e+03,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00]])

In [0]:
#Séparation des différents sets

X=np.zeros((len(headlines),length_max));

for  i,headline in enumerate(headlines):
   X[i,:]=encode(headline,dictionary);
    
print(X[0])

X_train = X[0:20000, :]
Y_train = labels[0:20000]
X_val = X[20000:25000,:]
Y_val = labels[20000:25000]
X_test = X[25000:, :]
Y_test = labels[25000:]

print(X_train.shape,X_val.shape,X_test.shape)

[3.140e+02 1.000e+00 6.560e+02 3.164e+03 2.329e+03 5.000e+01 3.730e+02
 9.500e+01 2.123e+03 5.000e+00 2.627e+03 8.262e+03 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
(20000, 39) (5000, 39) (1709, 39)


**Deep neural network**

On va maintenant créer un réseau de neurones qui prend en entier un vecteur d'indices (une phrase encodée) et qui sort une prédiction de sarcasme.

La première couche de notre réseau doit convertir un vecteur d'indice (vecteur 'one hot') en vecteur dense. Ceci se fait avec la couche tf.keras.layers.Embedding de keras.

Les couches suivantes sont à votre libre imagination.

La dernière couche est ne possède qu'un seul neurone de sortie avec une fonction d'activation sigmoide correspondant à une probabilité.

La fonction de coût appliquée est une crossentropie binaire.

Pensez également à couper votre ensemble entre train et val pour avoir des scores de généralisation !

In [0]:
model = keras.Sequential()
opt = keras.optimizers.Adam(lr=0.0001)#rmsprop
emb = layers.Embedding(input_dim=10001, output_dim=64,input_length=length_max)
model.add(emb); # 60*64 
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'));
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())
model.fit(x=X_train, y=Y_train,epochs=10,validation_data=(X_val,Y_val), verbose=1)
print(model.evaluate(x=X_test, y=Y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 39, 64)            640064    
_________________________________________________________________
flatten_18 (Flatten)         (None, 2496)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 2497      
Total params: 642,561
Trainable params: 642,561
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 8s 397us/step - loss: 0.6762 - acc: 0.5612 - val_loss: 0.6584 - val_acc: 0.5834
Epoch 2/10
20000/20000 [==============================] - 6s 324us/step - loss: 0.6153 - acc: 0.6708 - val_loss: 0.5693 - val_acc: 0.7816
Epoch 3/10
20000/20000 [==============================] - 7s 328us/step - loss: 0.5055 - acc: 0.8211 - val_loss: 0.4718 - val_acc: 0.8156
Epoch 4/10
20000/20000 [==============================] - 6s 324us/step - loss: 0.4149 - acc: 0.8606 - val_loss: 0.4111 - val_acc: 0.8446
Epoch 5/10
20000/20000 [==============================] - 6s 324us/step - loss: 0.3548 - acc: 0.8805 - val_loss: 0.3747 - val_acc: 0.8564
Epoch 6/10
20000/20000 [==============================] - 6s 325us/step - loss: 0.3126 - acc: 0.8941 - val_loss: 0.3515 - val_acc: 0.8614
Epoch 7/10
20000/20000 [==============================] - 6s 323us/step - loss: 0.2804 - acc: 0.9051 - val_loss: 0.3357 - v